In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import hvplot.pandas
import json
import holoviews as hv
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [ ]:
##Number of rides according to the weather? Ricardo

In [ ]:
##What hours of the day or in what weather are the peak requests? Lina

In [ ]:
##Which is the destination with the most rides?Kevin

In [ ]:
##Uber Fares depend on different times(day/night/day of week), duration(long/short)? Kevin

In [ ]:
##Which kind of city area(downtown, industry,financial central…) has the most uber request? Lina

In [ ]:
##Any relation between public transportation line and uber pickup and dropoff location? Kevin

In [ ]:
##Price surge? In relation to time of the day, rush hour or weather Zinny

In [ ]:
##Distribution of the rides by weekdays? Ricardo

In [ ]:
##Do people avoid rides on sunny days? Zinny

In [ ]:
##What are the earnings of Uber drivers based on factors such as time of day, location, or type of vehicle used?Lina